# setup

In [233]:
%pip install imageio
%pip install pyvirtualdisplay
%pip install tf-agents
%pip install pyglet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Imports

In [234]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

# Seed for PPO actor network
import tensorflow_probability as tfp

import tf_agents as ta

# PPO Agent
from tf_agents.agents.ppo import ppo_agent
from tf_agents.agents.ppo import ppo_actor_network
from tf_agents.networks import value_network
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_rnn_network
from tf_agents.agents.ppo import ppo_clip_agent

from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.sequential import Sequential

from tf_agents.drivers import py_driver

# old agent
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.networks import categorical_q_network
from tf_agents.networks import lstm_encoding_network

from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [235]:
env_name = "Acrobot-v2"  # @param {type:"string"}

train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

observation_tensor_spec = tensor_spec.from_spec(train_env.observation_spec())
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
time_step_tensor_spec = tensor_spec.from_spec(train_env.time_step_spec())

VersionNotFound: Environment version `v2` for `Acrobot` could not be found. `Acrobot` provides the versioned environments: [ `v1` ].
  In call to configurable 'load' (<function load at 0x131957310>)

In [ ]:
print('Observation: {0}'.format(observation_tensor_spec))
print('Action: {0}'.format(action_tensor_spec))
print('TimeStep: {0}'.format(time_step_tensor_spec))

# Agent

Network

In [ ]:

actor_fc_layers =(64,64)
value_fc_layers = (64,64)

actor_net = actor_distribution_network.ActorDistributionNetwork(
    observation_tensor_spec,
    action_tensor_spec,
    fc_layer_params=actor_fc_layers,
    activation_fn=tf.keras.activations.tanh,
    seed_stream_class=tfp.util.SeedStream(seed=None, salt='tf_agents_sequential_layers')
)

value_net = value_network.ValueNetwork(
    observation_tensor_spec,
    fc_layer_params=value_fc_layers,
    activation_fn=tf.keras.activations.tanh,
    kernel_initializer=tf.keras.initializers.Orthogonal()
)

In [236]:
learning_rate = 3e-4
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
epsilon = 0.2
discount = 0.99
gae = 0.95

train_step_counter = tf.Variable(0)

agent_ppo = ppo_clip_agent.PPOClipAgent(
    time_step_spec=time_step_tensor_spec,
    action_spec=action_tensor_spec,
    actor_net=actor_net,
    value_net=value_net,
    optimizer=optimizer,
    use_gae=True,
    importance_ratio_clipping=epsilon,
    # log_prob_clipping=epsilon,
    # gradient_clipping=epsilon,
    # value_clipping=epsilon,
    train_step_counter=train_step_counter,
    # compute_value_and_advantage_in_train=True,
    # update_normalizers_in_train=False,
    # num_epochs=10
)
agent_ppo.initialize()

agent_ppo.actor_net.summary()

agent_ppo._value_net.summary()


Model: "ActorDistributionNetwork"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncodingNetwork (EncodingNet multiple                  4608      
_________________________________________________________________
CategoricalProjectionNetwork multiple                  195       
Total params: 4,803
Trainable params: 4,803
Non-trainable params: 0
_________________________________________________________________
Model: "ValueNetwork"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncodingNetwork (EncodingNet multiple                  4608      
_________________________________________________________________
dense_79 (Dense)             multiple                  65        
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_eval_episodes = 10


def compute_avg_return(environment, policy, num_episodes):
    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        type(time_step)
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step, policy.get_initial_state(batch_size=1))
            time_step = environment.step(action_step.action)
            # print('time step: {0}'.format(time_step))
            episode_return += time_step.reward
        total_return += episode_return
    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]


random_policy = random_tf_policy.RandomTFPolicy(
    time_step_tensor_spec,
    action_tensor_spec,
    info_spec=time_step_tensor_spec)


def evaluate_policy(policy):
    for _ in range(1):
        avg = compute_avg_return(eval_env, policy, num_eval_episodes)
        print(avg)

# evaluate_policy(random_policy)
# 
# evaluate_policy(agent_ppo.policy)
# 
# evaluate_policy(agent_ppo.collect_policy)

In [ ]:
batch_size = 1
replay_buffer_size = 10000
replay_buffer_batch_size = 1

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent_ppo.collect_policy.trajectory_spec,
    batch_size=replay_buffer_batch_size,
    max_length=replay_buffer_size
)
rb_observer = replay_buffer.add_batch


# agent_ppo.collect_policy.trajectory_spec

In [ ]:
def collect_step(environment, policy):
    policy_state = policy._get_initial_state(environment.batch_size)
    time_step = environment.current_time_step()
    action_step = policy.action(time_step, policy_state)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    # print('Log Trajectory: {0}'.format(traj))
    replay_buffer.add_batch(traj)
    # print(next_time_step.reward)
    
collect_step(train_env, agent_ppo.collect_policy)


In [ ]:
initial_collect_steps = 10

for _ in range(initial_collect_steps):
    collect_step(train_env, agent_ppo.collect_policy)

In [232]:
n_step_update = 1
# batch_size = 64

dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2).prefetch(3)

iterator = iter(dataset)

In [237]:
num_iterations = 100
collect_steps_per_iteration = 10
eval_interval = 10
num_eval_episodes = 1

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent_ppo.train = common.function(agent_ppo.train)
agent_ppo.train_step_counter.assign(0)

avg_return = compute_avg_return(eval_env, agent_ppo.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):
    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(collect_steps_per_iteration):
        # collect_step(train_env, agent_ppo.collect_policy)
        collect_step(train_env, agent_ppo.collect_policy)
    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    # print('Xp: {0}'.format(experience))
    train_loss = agent_ppo.train(experience).loss

    step = agent_ppo.train_step_counter.numpy()

    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent_ppo.policy, num_eval_episodes)
        print('step = {0}: loss: {1} Average Return = {2:2f}'.format(step, train_loss,avg_return))
        returns.append(avg_return)


step = 50: loss: 21.132577896118164 Average Return = -500.000000
step = 100: loss: 21.123008728027344 Average Return = -500.000000
step = 150: loss: 20.894039154052734 Average Return = -500.000000
step = 200: loss: 21.188030242919922 Average Return = -500.000000
step = 250: loss: 21.15846061706543 Average Return = -500.000000
step = 300: loss: 21.14564323425293 Average Return = -500.000000
step = 350: loss: 21.222286224365234 Average Return = -500.000000
step = 400: loss: 21.28995132446289 Average Return = -500.000000
step = 450: loss: 21.16971778869629 Average Return = -500.000000
step = 500: loss: 25.47967529296875 Average Return = -500.000000
step = 550: loss: 20.967836380004883 Average Return = -500.000000
step = 600: loss: 21.14879608154297 Average Return = -500.000000
step = 650: loss: 20.96845245361328 Average Return = -500.000000
step = 700: loss: 20.86527442932129 Average Return = -500.000000
step = 750: loss: 20.84653663635254 Average Return = -500.000000
step = 800: loss: 20

KeyboardInterrupt: 

In [157]:
steps = 100

driver = py_driver.PyDriver(
    env=train_env,
    policy=agent_ppo.collect_policy,
    observers=[rb_observer],
    max_steps=steps,
)

time_step = train_env.reset()
driver.run(time_step)


(TimeStep(
 {'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
  'observation': <tf.Tensor: shape=(1, 6), dtype=float32, numpy=
 array([[0.9973923 , 0.07217032, 0.9038277 , 0.42789653, 0.1573971 ,
         0.23413408]], dtype=float32)>,
  'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.], dtype=float32)>,
  'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>}),
 ())

In [158]:
num_iterations = 1000
collect_steps_per_iteration = 10
num_eval_episodes = 1
eval_interval = 10

collect_driver = py_driver.PyDriver(
    train_env,
    agent_ppo.collect_policy,
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

    # Collect a few steps and save to the replay buffer.
    # time_step, _ = collect_driver.run(time_step)
    time_step, _ = collect_driver.run(time_step)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent_ppo.train(experience).loss
    # replay_buffer.clear()
    # print('Xp: {0}'.format(experience))
    # agent_ppo.train(experience)
    step = agent_ppo.train_step_counter.numpy()

    
    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent_ppo.policy, num_eval_episodes)
        print('step = {0}; Average Return= {1}; loss = {2}'.format(step, avg_return,train_loss))
        # print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)
        if(agent_ppo.collect_policy == agent_ppo.policy):
            print("True")

TypeError: 'NoneType' object is not callable

In [ ]:
steps = range(0, num_iterations + 1, 2)
# steps = range(0, num_iterations + 1, 10)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=0)